<a href="https://colab.research.google.com/github/glombardo/Research/blob/main/Mobility_Data_%2B_Observable_D3_%2B_Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset Details:

PlaceIQ sourced using SQL from Snowflake environment: for this demo the data is in comma delimmited format and shows details of locations visited before and/or after visiting a particular brand (specified the “BRAND” column). It contains 1M+ rows. The following attributes are in the dataset:
*   DEVIDE_ID: unique identifier of tracked person
*   BRAND: brand visited by DEVICE_ID
*   VISIT_TIMESTAMP: Visit timestamp to the BRAND by the DEVICE_ID
*   VISIT_DURATION: Visit duration to the Brand by the DEVICE_ID
*   STATE: Visit state to the BRANDby the Device ID
*   CITY: Visit city to the BRAND by the Device ID
*   ZIP_CODE: Visit zip code to the BRAND by the Device ID
*   LATITUDE: Visit Latitude to the BRAND by the Device ID
*   LONGITUDE: Visit Longitude to the BRAND by the Device ID
*   BRAND_COMP: competitor brand visited before or after visiting BRAND
*   TIMESTAMP_COMP: Visit timestamp to the BRAND_COMP by the DEVICE_ID
*   LATITUDE_COMP: Visit Latitude to the BRAND_COMP by the Device ID
*   LONGITUDE_COMP: Visit Longitude to the BRAND_COMP by the Device ID
*   PIG_CATEGORIES: category of BRAND_COMP
*   PRE_POST: Flag “BEFORE” or “AFTER” indicating whether BRAND_COMP was visited before or after BRAND.


# 1. Setup & Packages

In [1]:
import pandas as pd, numpy as np, polars as pl
import geopandas as gpd
from shapely.geometry import Point, LineString
import plotly.express as px, plotly.graph_objects as go
import folium, branca.colormap as cm
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import networkx as nx

In [3]:
# @title Default title text
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
path = "/content/drive/My Drive/dataprojects/vitamins_stores_placeIQ.csv"
df = pd.read_csv(path, engine="pyarrow", parse_dates=['VISIT_TIMESTAMP_LOCAL','TIMESTAMP_COMP'])
df.head(5)

,DEVICE_ID,BRAND,VISIT_TIMESTAMP_LOCAL,DURATION_SECONDS,STATE,CITY,ZIP_CODE,LATITUDE,LONGITUDE,PIQ_CATEGORIES,BRAND_COMP,CHAIN_NAME_COMP,TIMESTAMP_COMP,LATITUDE_COMP,LONGITUDE_COMP,SPATIAL_ID_COMP,PRE_POST
0,139686459255,CVS Pharmacy,2024-04-04 09:24:26+00:00,1400,NV,Las Vegas,89145,36.16,-115.28,"piq-paint-stores,piq-retail,piq-home-improvement",Sherwin-Williams,Sherwin-Williams,2024-02-20 08:51:26+00:00,36.16,-115.28,699644052754948096,BEFORE
1,139686459255,CVS Pharmacy,2024-04-04 09:24:26+00:00,1400,NV,Las Vegas,89145,36.16,-115.28,"piq-paint-stores,piq-retail,piq-home-improvement",Sherwin-Williams,Sherwin-Williams,2024-01-02 09:02:54+00:00,36.16,-115.28,699644052754948096,BEFORE
2,139686459255,CVS Pharmacy,2024-04-04 09:24:26+00:00,1400,NV,Las Vegas,89145,36.16,-115.28,"piq-convenience-stores-and-gas-stations,piq-re...",ARCO,ARCO,2024-01-08 20:28:48+00:00,36.16,-115.24,123183364578160640,BEFORE
3,139686459255,CVS Pharmacy,2024-04-04 09:24:26+00:00,1400,NV,Las Vegas,89145,36.16,-115.28,"piq-financial-services,piq-insurance",Farmers Insurance,Farmers Insurance,2024-01-09 12:57:12+00:00,36.16,-115.26,699644100656058368,BEFORE
4,139686459255,CVS Pharmacy,2024-04-04 09:24:26+00:00,1400,NV,Las Vegas,89145,36.16,-115.28,"piq-convenience-stores-and-gas-stations,piq-re...",AMPM,AMPM,2024-01-08 20:28:48+00:00,36.16,-115.24,123183364578160640,BEFORE


Dataset Details:

PlaceIQ data in comma delimmited format queried via Snowflake, showing details about locations visited before and/or after visiting a particular brand (specified the “BRAND” column). It contains 1M+ rows. The following attributes are in the dataset:
*   DEVIDE_ID: unique identifier of tracked person
*   BRAND: brand visited by DEVICE_ID
*   VISIT_TIMESTAMP: Visit timestamp to the BRAND by the DEVICE_ID
*   VISIT_DURATION: Visit duration to the Brand by the DEVICE_ID
*   STATE: Visit state to the BRANDby the Device ID
*   CITY: Visit city to the BRAND by the Device ID
*   ZIP_CODE: Visit zip code to the BRAND by the Device ID
*   LATITUDE: Visit Latitude to the BRAND by the Device ID
*   LONGITUDE: Visit Longitude to the BRAND by the Device ID
*   BRAND_COMP: competitor brand visited before or after visiting BRAND
*   TIMESTAMP_COMP: Visit timestamp to the BRAND_COMP by the DEVICE_ID
*   LATITUDE_COMP: Visit Latitude to the BRAND_COMP by the Device ID
*   LONGITUDE_COMP: Visit Longitude to the BRAND_COMP by the Device ID
*   PIG_CATEGORIES: category of BRAND_COMP
*   PRE_POST: Flag “BEFORE” or “AFTER” indicating whether BRAND_COMP was visited before or after BRAND.


# Geo-Timeline Visualization

This D3 Geo-Timeline Visualization will take a file with columns "0", "1", and "date" representing longitude, latitude, and date (at the day level) respectively.

We need to create new dataframe copying df and keep columns LATITUDE_COMP, LONGITUDE_COMP and TIMESTAMP_COMP.

Then, transform TIMESTAMP_COMP so it is called 'date' and is in format "mm/dd/yy" and group by LATITUDE_COMP, LATITUDE_COMP.

And.... convert dates to '%Y-%m-%dT%H:%MZ'

Finally, Call LONGITUDE_COMP "0" and LATITUDE_COMP "1" to make our lives easier.



In [ ]:
# need to create new dataframe copying df and keep columns LATITUDE_COMP, LONGITUDE_COMP and TIMESTAMP_COMP.
#Then, transform TIMESTAMP_COMP so it is called 'date' and is in format "mm/dd/yy" and group by LATITUDE_COMP, LATITUDE_COMP.
#Finally, Call LONGITUDE_COMP "0" and LATITUDE_COMP "1".

df_geo = df[['LONGITUDE_COMP', 'LATITUDE_COMP', 'TIMESTAMP_COMP']].copy()

#for df_geo we will keep only the rows with date occurring in February 2024 since we can only handle 50MB with this plot


In [ ]:

df_geo['TIMESTAMP_COMP'] = pd.to_datetime(df_geo['TIMESTAMP_COMP'], utc=True)
df_geo = df_geo[(df_geo['TIMESTAMP_COMP'].dt.year == 2024) & (df_geo['TIMESTAMP_COMP'].dt.month == 2)]
df_geo['date']= df_geo['TIMESTAMP_COMP'].dt.strftime('%Y-%m-%dT%H:%MZ')
df_geo = df_geo.drop(columns=['TIMESTAMP_COMP'])
df_geo = df_geo.rename(columns={'LONGITUDE_COMP': '0', 'LATITUDE_COMP': '1'})
df_geo.head()

,0,1,date
0,-115.28,36.16,2024-02-20T08:51Z
6,-158.03,21.50,2024-02-27T06:04Z
8,-77.51,38.30,2024-02-08T12:02Z
11,-77.51,38.30,2024-02-08T08:30Z
14,-77.52,38.30,2024-02-08T11:46Z


In [ ]:
df_geo_filtered = df_geo
df_geo_filtered.head()

,0,1,date
0,-115.28,36.16,2024-02-20T08:51Z
6,-158.03,21.50,2024-02-27T06:04Z
8,-77.51,38.30,2024-02-08T12:02Z
11,-77.51,38.30,2024-02-08T08:30Z
14,-77.52,38.30,2024-02-08T11:46Z


In [ ]:
df_geo_filtered.to_csv('vitamin_mobility_geo_february2024_final.tsv', sep='\t', index=False)

#Sankey Diagram File

This D3 Sankey Diagram will take a file with columns "source", "target" and "volume". This requires:


*   Follow the following rules: 1) 'source' can be BRAND or BRAND_COMP 3) BRAND_COMP is a 'TARGET' of BRAND (when BRAND is a source) if PRE_POST = 'AFTER' 3) BRAND_COMP is a 'source' of BRAND (when BRAND is a target) if PRE_POST = 'before'. 4) 'value' is the aggregate distinct counts of DEVICE_IDs for each combination of 'source' and 'target'
*   A directed acyclic graph representation. Therefore, we will need to prune the lowest-volume edge from each cycle until the graph is acyclic. Note than removing pair-wise bidirectional links is not enough because it doesn catch longer cycles like A → B, B → C, C → A.

The functions below take care of this.

In [ ]:
#source-target pairs (distinct DEVICE_ID count)
def build_source_target_df(df: pd.DataFrame) -> pd.DataFrame:
    logic_after = df["PRE_POST"].str.upper().eq("AFTER")

    df = df.copy()
    df["source"] = np.where(logic_after, df["BRAND"],      df["BRAND_COMP"])
    df["target"] = np.where(logic_after, df["BRAND_COMP"], df["BRAND"])
    df = df.dropna(subset=["source", "target"])

    return (
        df.groupby(["source", "target"], observed=True)["DEVICE_ID"]
          .nunique()
          .reset_index(name="value")
          .sort_values("value", ascending=False)
          .reset_index(drop=True)
    )

#collapse duplicate rows & drop simple A↔B pairs
def tidy_links(flow_df: pd.DataFrame) -> pd.DataFrame:
    flow_df = (
        flow_df.groupby(["source", "target"], as_index=False, observed=True)["value"]
               .sum()
    )
    flow_df["pair"] = flow_df.apply(
        lambda r: tuple(sorted((r["source"], r["target"]))), axis=1
    )
    flow_df = (
        flow_df.sort_values("value", ascending=False)
               .drop_duplicates("pair", keep="first")
               .drop(columns="pair")
               .reset_index(drop=True)
    )
    return flow_df

#Remove longer cycles so graph is a DAG
def drop_circular_links(flow_df: pd.DataFrame) -> pd.DataFrame:
    G = nx.DiGraph()
    keep = []
    for src, tgt, val in flow_df.sort_values("value", ascending=False).itertuples(False):
        G.add_edge(src, tgt, weight=val)
        if nx.is_directed_acyclic_graph(G):
            keep.append((src, tgt, val))
        else:
            G.remove_edge(src, tgt)
    return pd.DataFrame(keep, columns=["source", "target", "value"])



In [ ]:
links_df = build_source_target_df(df)
links_df = tidy_links(links_df)
links_df = drop_circular_links(links_df)

links_df_2 = links_df.groupby(['source', 'target'], as_index=False, observed=True)['value'].sum()
links_df_2.sort_values('value', ascending=False).head()
links_df_2 = links_df_2[links_df_2['value'] > 300].copy()
links_df_2['value'] = links_df_2['value'].apply(lambda x: min(x, 200))

links_df_2.to_csv('vitamin_mobility_sankey.csv', index=False)

#Association Rule Mining

In [5]:
df['BRAND'].unique()

array(['CVS Pharmacy', 'Walgreens', 'The Vitamin Shoppe', 'GNC', 'Cvs',
       'Signco Graphics',
       'First Coast Veterinary Specialists And Emergency (fcvs)', 'CVS',
       'Kcvs', 'Designcor LLC', '6th Avenue Gnc - Inside',
       'Kenneth A Bruecker DVM MS Dacvs', 'Lavalle Agncy',
       'Signcrafters Inc', 'Signco USA', 'Delgado Travel Agncy',
       'Signcentral', 'CVS Tech Inc', 'Daproza Insurnc Agncy',
       'Ccreativedesigncom', 'Cabinet Designcenter LLC',
       'Audi Independent Service-By CVS', 'Kcvs Radio 90.7 FM',
       'Duensing Contraction & Designcraft Homes',
       'Reyes Ins Roxanne Agncy', 'Curtis J Vernon Ins Agncy',
       'Banyantitle Agncy', 'Signco', 'Designcraft Builders Inc',
       "Pierc's4wdresurrection P La Piercvs Wd Resurrection",
       'Keystone Ins Agncy', 'Nelson H Priddy II DVM Dacvs',
       'Varrone Agncy', 'Gncm Production', 'Murphy Ins Agncy',
       'La-Tex Billing Scvs LLC', 'Signcom Inc', 'Cvs Pharmacy',
       'Cvs/Granite', 'A Signco

In [6]:
def basket_after_before(d_after, d_before, support):
  basket_after = (d_after[['DEVICE_ID','BRAND_COMP']]
            .drop_duplicates()
            .assign(flag=1)
            .pivot_table(index='DEVICE_ID',
                        columns='BRAND_COMP',
                        values='flag',
                        fill_value=0))
  freq_after = apriori(basket_after, min_support=support, use_colnames=True) #min_support=0.04
  rules_after = association_rules(freq_after, metric='lift', min_threshold=1.1).sort_values('confidence', ascending=False)

  basket_before = (d_before[['DEVICE_ID','BRAND_COMP']]
            .drop_duplicates()
            .assign(flag=1)
            .pivot_table(index='DEVICE_ID',
                        columns='BRAND_COMP',
                        values='flag',
                        fill_value=0))
  freq_before = apriori(basket_before, min_support=support, use_colnames=True) #min_support=0.04
  rules_before = association_rules(freq_before, metric='lift', min_threshold=1.1).sort_values('confidence', ascending=False)

  rules_after['timing'] = 'AFTER'
  rules_before['timing'] = 'BEFORE'
  rules_combined = pd.concat([rules_after, rules_before]).reset_index(drop=True)

  return rules_combined

In [7]:
target_brands = ['CVS Pharmacy', 'Walgreens']#, 'The Vitamin Shoppe', 'GNC', 'Cvs']
df_filtered = df[df['BRAND'].isin(target_brands)].copy()

In [11]:
current_after = df_filtered[(df_filtered['PRE_POST'] == 'AFTER') & (df_filtered['BRAND'] == target_brands[0])]
current_before = df_filtered[(df_filtered['PRE_POST'] == 'BEFORE') & (df_filtered['BRAND'] == target_brands[0])]
all_combined = basket_after_before(current_after, current_before, 0.01)
all_combined['brand'] = target_brands[0]
all_combined

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,timing,brand
0,(AMPM),(ARCO),0.010862,0.019230,0.010105,0.930233,48.373620,1.0,0.009896,14.057701,0.990082,0.505529,0.928865,0.727842,AFTER,CVS Pharmacy
1,(ARCO),(AMPM),0.019230,0.010862,0.010105,0.525452,48.373620,1.0,0.009896,2.084377,0.998529,0.505529,0.520240,0.727842,AFTER,CVS Pharmacy
2,(KFC),(Taco Bell),0.041634,0.096388,0.018820,0.452029,4.689697,1.0,0.014807,1.649014,0.820946,0.157881,0.393577,0.323639,AFTER,CVS Pharmacy
3,(Pizza Hut),(Taco Bell),0.028719,0.096388,0.010373,0.361187,3.747239,1.0,0.007605,1.414519,0.754814,0.090409,0.293046,0.234402,AFTER,CVS Pharmacy
4,(Popeyes Famous Fried Chicken),(Taco Bell),0.034971,0.096388,0.011510,0.329120,3.414542,1.0,0.008139,1.346905,0.732760,0.096035,0.257557,0.224265,AFTER,CVS Pharmacy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,(Walgreens),(BP Oil),0.102200,0.055211,0.010869,0.106349,1.926245,1.0,0.005226,1.057224,0.535593,0.074169,0.054127,0.151606,BEFORE,CVS Pharmacy
196,(Shell Station),(KFC),0.106629,0.041334,0.011310,0.106070,2.566145,1.0,0.006903,1.072417,0.683155,0.082765,0.067527,0.189848,BEFORE,CVS Pharmacy
197,(Shell Station),(marathon),0.106629,0.042854,0.011294,0.105917,2.471549,1.0,0.006724,1.070533,0.666460,0.081727,0.065886,0.184728,BEFORE,CVS Pharmacy
198,(Shell Station),(Dollar General),0.106629,0.060588,0.010902,0.102238,1.687435,1.0,0.004441,1.046393,0.456009,0.069741,0.044336,0.141084,BEFORE,CVS Pharmacy


In [12]:
for brand in target_brands[1:]:
  current_after = df_filtered[(df_filtered['PRE_POST'] == 'AFTER') & (df_filtered['BRAND'] == brand)]
  current_before = df_filtered[(df_filtered['PRE_POST'] == 'BEFORE') & (df_filtered['BRAND'] == brand)]
  current_rules = basket_after_before(current_after, current_before, 0.04)
  current_rules['brand'] = brand
  current_rules
  if all_combined.empty:
    all_combined = current_rules
  else:
    all_combined = pd.concat([current_rules, all_combined]).reset_index(drop=True)

all_combined

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/fpcommon.py:161: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(
/tmp/ipython-input-12-2741398574.py:10: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_combined = pd.concat([current_rules, all_combined]).reset_index(drop=True)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,timing,brand
0,(AMPM),(ARCO),0.010862,0.019230,0.010105,0.930233,48.373620,1.0,0.009896,14.057701,0.990082,0.505529,0.928865,0.727842,AFTER,CVS Pharmacy
1,(ARCO),(AMPM),0.019230,0.010862,0.010105,0.525452,48.373620,1.0,0.009896,2.084377,0.998529,0.505529,0.520240,0.727842,AFTER,CVS Pharmacy
2,(KFC),(Taco Bell),0.041634,0.096388,0.018820,0.452029,4.689697,1.0,0.014807,1.649014,0.820946,0.157881,0.393577,0.323639,AFTER,CVS Pharmacy
3,(Pizza Hut),(Taco Bell),0.028719,0.096388,0.010373,0.361187,3.747239,1.0,0.007605,1.414519,0.754814,0.090409,0.293046,0.234402,AFTER,CVS Pharmacy
4,(Popeyes Famous Fried Chicken),(Taco Bell),0.034971,0.096388,0.011510,0.329120,3.414542,1.0,0.008139,1.346905,0.732760,0.096035,0.257557,0.224265,AFTER,CVS Pharmacy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,(Walgreens),(BP Oil),0.102200,0.055211,0.010869,0.106349,1.926245,1.0,0.005226,1.057224,0.535593,0.074169,0.054127,0.151606,BEFORE,CVS Pharmacy
196,(Shell Station),(KFC),0.106629,0.041334,0.011310,0.106070,2.566145,1.0,0.006903,1.072417,0.683155,0.082765,0.067527,0.189848,BEFORE,CVS Pharmacy
197,(Shell Station),(marathon),0.106629,0.042854,0.011294,0.105917,2.471549,1.0,0.006724,1.070533,0.666460,0.081727,0.065886,0.184728,BEFORE,CVS Pharmacy
198,(Shell Station),(Dollar General),0.106629,0.060588,0.010902,0.102238,1.687435,1.0,0.004441,1.046393,0.456009,0.069741,0.044336,0.141084,BEFORE,CVS Pharmacy


In [13]:
# prompt: export dataframe all_combined to excel

all_combined.to_excel("all_combined.xlsx", index=False)